## 1. Environment Setup

In [ ]:
# Install required packages
# !pip install tensorflow opencv-python mediapipe numpy pandas scikit-learn matplotlib seaborn

In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization,
    Conv1D, MaxPooling1D, GlobalAveragePooling1D,
    LSTM, GRU, Bidirectional, Reshape, Flatten, concatenate,
    MultiHeadAttention, LayerNormalization
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# MediaPipe & OpenCV
import cv2
import mediapipe as mp

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 2. Configuration

In [ ]:
# ============================================
# ALL 111 SIGN LANGUAGE GESTURES
# ============================================

SIGNS = {
    # Alphabet (26 signs)
    'A': 'Letter A', 'B': 'Letter B', 'C': 'Letter C', 'D': 'Letter D',
    'E': 'Letter E', 'F': 'Letter F', 'G': 'Letter G', 'H': 'Letter H',
    'I': 'Letter I', 'J': 'Letter J', 'K': 'Letter K', 'L': 'Letter L',
    'M': 'Letter M', 'N': 'Letter N', 'O': 'Letter O', 'P': 'Letter P',
    'Q': 'Letter Q', 'R': 'Letter R', 'S': 'Letter S', 'T': 'Letter T',
    'U': 'Letter U', 'V': 'Letter V', 'W': 'Letter W', 'X': 'Letter X',
    'Y': 'Letter Y', 'Z': 'Letter Z',
    
    # Numbers (10 signs)
    '0': 'Number 0', '1': 'Number 1', '2': 'Number 2', '3': 'Number 3',
    '4': 'Number 4', '5': 'Number 5', '6': 'Number 6', '7': 'Number 7',
    '8': 'Number 8', '9': 'Number 9',
    
    # Greetings (9 signs)
    'HELLO': 'Hello/Hi', 'GOODBYE': 'Goodbye', 'THANK_YOU': 'Thank You',
    'PLEASE': 'Please', 'SORRY': 'Sorry', 'WELCOME': 'Welcome',
    'GOOD_MORNING': 'Good Morning', 'GOOD_NIGHT': 'Good Night',
    'NICE_TO_MEET': 'Nice to Meet You',
    
    # Responses (9 signs)
    'YES': 'Yes', 'NO': 'No', 'MAYBE': 'Maybe', 'OK': 'OK',
    'DONT_KNOW': "I Don't Know", 'UNDERSTAND': 'I Understand',
    'DONT_UNDERSTAND': "I Don't Understand", 'AGREE': 'Agree', 'DISAGREE': 'Disagree',
    
    # Actions (26 signs)
    'EAT': 'Eat', 'DRINK': 'Drink', 'SLEEP': 'Sleep', 'WAKE_UP': 'Wake Up',
    'WALK': 'Walk', 'RUN': 'Run', 'STOP': 'Stop', 'GO': 'Go', 'COME': 'Come',
    'WAIT': 'Wait', 'SIT': 'Sit', 'STAND': 'Stand', 'HELP': 'Help',
    'WORK': 'Work', 'PLAY': 'Play', 'LEARN': 'Learn', 'TEACH': 'Teach',
    'READ': 'Read', 'WRITE': 'Write', 'LISTEN': 'Listen', 'SPEAK': 'Speak',
    'WATCH': 'Watch', 'OPEN': 'Open', 'CLOSE': 'Close', 'GIVE': 'Give', 'TAKE': 'Take',
    
    # Emotions (10 signs)
    'HAPPY': 'Happy', 'SAD': 'Sad', 'ANGRY': 'Angry', 'SCARED': 'Scared',
    'SURPRISED': 'Surprised', 'TIRED': 'Tired', 'EXCITED': 'Excited',
    'BORED': 'Bored', 'CONFUSED': 'Confused', 'PROUD': 'Proud',
    
    # Questions (8 signs)
    'WHAT': 'What?', 'WHERE': 'Where?', 'WHEN': 'When?', 'WHO': 'Who?',
    'WHY': 'Why?', 'HOW': 'How?', 'HOW_MUCH': 'How Much?', 'HOW_MANY': 'How Many?',
    
    # Time (7 signs)
    'NOW': 'Now', 'LATER': 'Later', 'BEFORE': 'Before', 'AFTER': 'After',
    'TODAY': 'Today', 'TOMORROW': 'Tomorrow', 'YESTERDAY': 'Yesterday',
    
    # People (10 signs)
    'I_ME': 'I/Me', 'YOU': 'You', 'HE_SHE': 'He/She', 'WE': 'We', 'THEY': 'They',
    'FRIEND': 'Friend', 'FAMILY': 'Family', 'MOTHER': 'Mother',
    'FATHER': 'Father', 'CHILD': 'Child'
}

SIGN_LABELS = list(SIGNS.keys())
NUM_SIGNS = len(SIGNS)

# Sign Categories
SIGN_CATEGORIES = {
    'Alphabet': [chr(i) for i in range(65, 91)],
    'Numbers': [str(i) for i in range(10)],
    'Greetings': ['HELLO', 'GOODBYE', 'THANK_YOU', 'PLEASE', 'SORRY', 'WELCOME', 
                  'GOOD_MORNING', 'GOOD_NIGHT', 'NICE_TO_MEET'],
    'Responses': ['YES', 'NO', 'MAYBE', 'OK', 'DONT_KNOW', 'UNDERSTAND', 
                  'DONT_UNDERSTAND', 'AGREE', 'DISAGREE'],
    'Actions': ['EAT', 'DRINK', 'SLEEP', 'WAKE_UP', 'WALK', 'RUN', 'STOP', 'GO', 'COME',
                'WAIT', 'SIT', 'STAND', 'HELP', 'WORK', 'PLAY', 'LEARN', 'TEACH',
                'READ', 'WRITE', 'LISTEN', 'SPEAK', 'WATCH', 'OPEN', 'CLOSE', 'GIVE', 'TAKE'],
    'Emotions': ['HAPPY', 'SAD', 'ANGRY', 'SCARED', 'SURPRISED', 'TIRED', 
                 'EXCITED', 'BORED', 'CONFUSED', 'PROUD'],
    'Questions': ['WHAT', 'WHERE', 'WHEN', 'WHO', 'WHY', 'HOW', 'HOW_MUCH', 'HOW_MANY'],
    'Time': ['NOW', 'LATER', 'BEFORE', 'AFTER', 'TODAY', 'TOMORROW', 'YESTERDAY'],
    'People': ['I_ME', 'YOU', 'HE_SHE', 'WE', 'THEY', 'FRIEND', 'FAMILY', 
               'MOTHER', 'FATHER', 'CHILD']
}

print(f"✅ Total Signs: {NUM_SIGNS}")
print(f"📁 Categories: {len(SIGN_CATEGORIES)}")
for cat, signs in SIGN_CATEGORIES.items():
    print(f"   • {cat}: {len(signs)} signs")

In [ ]:
# ============================================
# PROJECT CONFIGURATION
# ============================================

# Paths
DATA_DIR = 'data'
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
MODELS_DIR = 'models'
OUTPUTS_DIR = 'outputs'
DATASET_CSV = os.path.join(RAW_DATA_DIR, 'sign_dataset.csv')

# Create directories
for d in [RAW_DATA_DIR, PROCESSED_DATA_DIR, MODELS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)

# MediaPipe Settings
MP_MAX_HANDS = 2
MP_MIN_DETECTION_CONF = 0.7
MP_MIN_TRACKING_CONF = 0.5

# Features
NUM_LANDMARKS = 21
COORDS_PER_LANDMARK = 3
FEATURES_PER_HAND = NUM_LANDMARKS * COORDS_PER_LANDMARK  # 63
TOTAL_FEATURES = FEATURES_PER_HAND * MP_MAX_HANDS  # 126

# Training Settings
EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 0.001
TEST_SPLIT = 0.15
VALIDATION_SPLIT = 0.15

# Detection Settings
CONFIDENCE_THRESHOLD = 0.6
PREDICTION_BUFFER_SIZE = 5

print("✅ Configuration loaded")
print(f"📊 Features per sample: {TOTAL_FEATURES}")

## 3. Data Collection (Optional)

In [ ]:
class HandDetector:
    """MediaPipe hand detection wrapper"""
    
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.mp_draw = mp.solutions.drawing_utils
        
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=MP_MAX_HANDS,
            min_detection_confidence=MP_MIN_DETECTION_CONF,
            min_tracking_confidence=MP_MIN_TRACKING_CONF
        )
    
    def detect(self, frame):
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return self.hands.process(rgb)
    
    def extract_landmarks(self, frame):
        results = self.detect(frame)
        
        left_hand = np.zeros(FEATURES_PER_HAND)
        right_hand = np.zeros(FEATURES_PER_HAND)
        hand_detected = False
        
        if results.multi_hand_landmarks:
            hand_detected = True
            
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                if idx >= len(results.multi_handedness):
                    continue
                
                hand_label = results.multi_handedness[idx].classification[0].label
                
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                
                landmarks = np.array(landmarks)
                
                if hand_label == 'Left':
                    left_hand = landmarks
                else:
                    right_hand = landmarks
        
        features = np.concatenate([left_hand, right_hand])
        return features, results, hand_detected
    
    def draw_landmarks(self, frame, results):
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(
                    frame, hand_landmarks,
                    self.mp_hands.HAND_CONNECTIONS
                )
        return frame
    
    def release(self):
        self.hands.close()

print("✅ HandDetector class defined")

In [ ]:
def create_synthetic_dataset(num_samples=1000, signs_to_use=None):
    """
    Create synthetic dataset for testing/demonstration
    In real usage, collect actual hand landmark data
    """
    if signs_to_use is None:
        signs_to_use = SIGN_LABELS[:20]  # Use first 20 signs
    
    print(f"Creating synthetic dataset with {len(signs_to_use)} signs...")
    
    data = []
    samples_per_sign = num_samples // len(signs_to_use)
    
    for sign in signs_to_use:
        # Generate base pattern for this sign
        base_pattern = np.random.randn(TOTAL_FEATURES) * 0.5
        
        for _ in range(samples_per_sign):
            # Add variation to base pattern
            noise = np.random.randn(TOTAL_FEATURES) * 0.1
            features = base_pattern + noise
            
            row = {
                'sign': sign,
                'timestamp': datetime.now().isoformat()
            }
            for i, f in enumerate(features):
                row[f'feature_{i}'] = f
            
            data.append(row)
    
    df = pd.DataFrame(data)
    df.to_csv(DATASET_CSV, index=False)
    
    print(f"✅ Created {len(df)} samples for {df['sign'].nunique()} signs")
    print(f"💾 Saved to: {DATASET_CSV}")
    
    return df

# Create sample dataset
# Uncomment to create synthetic data for testing
# df = create_synthetic_dataset(num_samples=2000, signs_to_use=SIGN_LABELS[:30])

## 4. Data Loading & Exploration

In [ ]:
# Load or create dataset
if os.path.exists(DATASET_CSV):
    df = pd.read_csv(DATASET_CSV)
    print(f"✅ Loaded existing dataset: {len(df)} samples")
else:
    print("⚠️ No dataset found. Creating synthetic data for demonstration...")
    df = create_synthetic_dataset(num_samples=3000, signs_to_use=SIGN_LABELS[:30])

print(f"\n📊 Dataset Summary:")
print(f"   Total samples: {len(df)}")
print(f"   Total signs: {df['sign'].nunique()}")
print(f"   Features: {len([c for c in df.columns if c.startswith('feature_')])}")

In [ ]:
# Visualize distribution
plt.figure(figsize=(14, 6))

sign_counts = df['sign'].value_counts()

if len(sign_counts) > 30:
    sign_counts[:30].plot(kind='bar', color='steelblue')
    plt.title('Dataset Distribution (Top 30 Signs)', fontsize=14, fontweight='bold')
else:
    sign_counts.plot(kind='bar', color='steelblue')
    plt.title('Dataset Distribution', fontsize=14, fontweight='bold')

plt.xlabel('Sign')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"\nSamples per sign:")
print(sign_counts)

## 5. Data Preprocessing

In [ ]:
# Extract features and labels
feature_cols = [col for col in df.columns if col.startswith('feature_')]

X = df[feature_cols].values
y = df['sign'].values

# Handle missing values
X = np.nan_to_num(X, nan=0.0)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_

print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

In [ ]:
# Data Augmentation
def augment_data(X, y, augmentation_factor=2):
    """Augment data with noise and scaling"""
    print(f"Augmenting data (factor: {augmentation_factor})...")
    
    augmented_X = [X]
    augmented_y = [y]
    
    for i in range(augmentation_factor - 1):
        # Add Gaussian noise
        noise = np.random.normal(0, 0.02, X.shape)
        X_noisy = X + noise
        
        # Add scaling variation
        scale = np.random.uniform(0.95, 1.05, X.shape)
        X_scaled = X * scale
        
        augmented_X.extend([X_noisy, X_scaled])
        augmented_y.extend([y, y])
    
    X_aug = np.vstack(augmented_X)
    y_aug = np.hstack(augmented_y)
    
    print(f"Augmented: {len(X)} → {len(X_aug)} samples")
    return X_aug, y_aug

X_aug, y_aug = augment_data(X, y_encoded, augmentation_factor=2)

In [ ]:
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_aug)

print(f"Feature range: [{X_scaled.min():.2f}, {X_scaled.max():.2f}]")

In [ ]:
# Split data
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_scaled, y_aug, test_size=TEST_SPLIT, random_state=42, stratify=y_aug
)

val_ratio = VALIDATION_SPLIT / (1 - TEST_SPLIT)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=val_ratio, random_state=42, stratify=y_trainval
)

print(f"\nData Split:")
print(f"  Train: {X_train.shape[0]} samples")
print(f"  Val:   {X_val.shape[0]} samples")
print(f"  Test:  {X_test.shape[0]} samples")

## 6. Model Building

In [ ]:
def create_cnn_model(num_features, num_classes):
    """1D CNN Model"""
    inputs = Input(shape=(num_features,))
    
    x = Reshape((num_features, 1))(inputs)
    
    x = Conv1D(256, 3, activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv1D(64, 3, activation='relu', padding='same')(x)
    x = GlobalAveragePooling1D()(x)
    
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs, name='CNN_Model')
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def create_lstm_model(num_features, num_classes):
    """Bidirectional LSTM Model"""
    inputs = Input(shape=(num_features,))
    
    x = Reshape((42, 3))(inputs)  # 42 landmarks * 3 coords
    
    x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001)))(x)
    x = Dropout(0.4)(x)
    
    x = Bidirectional(LSTM(64, return_sequences=False))(x)
    x = Dropout(0.4)(x)
    
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.4)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs, name='LSTM_Model')
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def create_hybrid_model(num_features, num_classes):
    """Hybrid CNN + LSTM + GRU Model"""
    inputs = Input(shape=(num_features,))
    
    # CNN Branch
    x_cnn = Reshape((num_features, 1))(inputs)
    x_cnn = Conv1D(64, 3, activation='relu', padding='same')(x_cnn)
    x_cnn = BatchNormalization()(x_cnn)
    x_cnn = MaxPooling1D(2)(x_cnn)
    x_cnn = Conv1D(128, 3, activation='relu', padding='same')(x_cnn)
    x_cnn = GlobalAveragePooling1D()(x_cnn)
    
    # LSTM Branch
    x_lstm = Reshape((42, 3))(inputs)
    x_lstm = LSTM(64, return_sequences=True)(x_lstm)
    x_lstm = LSTM(64, return_sequences=False)(x_lstm)
    
    # GRU Branch
    x_gru = Reshape((42, 3))(inputs)
    x_gru = GRU(64, return_sequences=False)(x_gru)
    
    # Concatenate
    x = concatenate([x_cnn, x_lstm, x_gru])
    
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs, name='Hybrid_Model')
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("✅ Model builders defined")

In [ ]:
# Select and create model
MODEL_TYPE = 'hybrid'  # Options: 'cnn', 'lstm', 'hybrid'

model_builders = {
    'cnn': create_cnn_model,
    'lstm': create_lstm_model,
    'hybrid': create_hybrid_model
}

model = model_builders[MODEL_TYPE](X_train.shape[1], num_classes)
model.summary()

## 7. Training

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
]

print("Starting training...")

In [ ]:
# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUTS_DIR, 'training_history.png'), dpi=150)
plt.show()

## 8. Evaluation

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

print(f"\n📊 Test Results:")
print(f"   Loss: {test_loss:.4f}")
print(f"   Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

In [ ]:
# Predictions
y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)

# Classification report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names[:len(np.unique(y_test))]))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(12, 10))

if num_classes > 20:
    sns.heatmap(cm, cmap='Blues', cbar=True)
    plt.title(f'Confusion Matrix ({num_classes} classes)', fontsize=14, fontweight='bold')
else:
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')

plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUTS_DIR, 'confusion_matrix.png'), dpi=150)
plt.show()

In [ ]:
# Save model
model_path = os.path.join(MODELS_DIR, f'{MODEL_TYPE}_sign_model.h5')
model.save(model_path)
print(f"✅ Model saved: {model_path}")

# Save preprocessors
import pickle

with open(os.path.join(MODELS_DIR, 'label_encoder.pkl'), 'wb') as f:
    pickle.dump(label_encoder, f)

with open(os.path.join(MODELS_DIR, 'scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)

print("✅ Preprocessors saved")

## 9. Real-time Detection (Run in separate Python script)

In [ ]:
# Real-time detection code
# Note: Run this in a separate Python script for best results

def run_detection():
    """Run real-time sign language detection"""
    
    # Load model and preprocessors
    loaded_model = tf.keras.models.load_model(model_path)
    
    with open(os.path.join(MODELS_DIR, 'label_encoder.pkl'), 'rb') as f:
        loaded_encoder = pickle.load(f)
    
    with open(os.path.join(MODELS_DIR, 'scaler.pkl'), 'rb') as f:
        loaded_scaler = pickle.load(f)
    
    detector = HandDetector()
    
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print("\n🎥 Starting detection. Press 'q' to quit.")
    
    from collections import deque
    prediction_buffer = deque(maxlen=5)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        features, results, hand_detected = detector.extract_landmarks(frame)
        frame = detector.draw_landmarks(frame, results)
        
        sign_name = "No hand detected"
        confidence = 0.0
        
        if hand_detected:
            # Preprocess
            features_scaled = loaded_scaler.transform(features.reshape(1, -1))
            
            # Predict
            predictions = loaded_model.predict(features_scaled, verbose=0)
            class_idx = np.argmax(predictions[0])
            confidence = predictions[0][class_idx]
            
            prediction_buffer.append((class_idx, confidence))
            
            # Get smoothed prediction
            from collections import Counter
            preds = [p[0] for p in prediction_buffer]
            most_common = Counter(preds).most_common(1)[0][0]
            avg_conf = np.mean([c for p, c in prediction_buffer if p == most_common])
            
            if avg_conf >= CONFIDENCE_THRESHOLD:
                sign_name = loaded_encoder.inverse_transform([most_common])[0]
                confidence = avg_conf
        
        # Draw UI
        cv2.rectangle(frame, (0, 0), (640, 100), (50, 50, 50), -1)
        cv2.putText(frame, f"Sign: {sign_name}", (20, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
        cv2.putText(frame, f"Confidence: {confidence*100:.1f}%", (20, 85),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
        
        cv2.imshow('Sign Language Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    detector.release()

# Uncomment to run detection
# run_detection()

## 📊 Summary

This notebook provides a complete pipeline for:

1. **111 Sign Language Gestures** - A-Z, 0-9, and 75 custom signs
2. **Data Collection** - Using MediaPipe hand landmarks (126 features)
3. **Preprocessing** - Augmentation, normalization, train-test split
4. **Multiple Models** - CNN, LSTM, Hybrid architectures
5. **Training** - With callbacks for early stopping and learning rate reduction
6. **Evaluation** - Accuracy, classification report, confusion matrix
7. **Real-time Detection** - Live webcam inference

For production use:
- Collect real hand landmark data using the `src/data_collection.py` module
- Train with more epochs and larger dataset
- Use the `main.py` script for interactive menu-driven access